# Step by step
I will start from a basic, almost empty model, and work to gradually improve it.
# IMDB
The competition rules state that "You can collect other publicly available data to use in your model predictions, but in the spirit of this competition, use only data that would have been available before a movie's release.". I have (in a separate Kernel) read the following data from imdb: year, budget, runtime, country, etc. - all data that was known before the release, and will try to merge it with the training data.

# In this iteration

fix budget? entries with 0? make it log? Some are in millions - need to multiple by 1000000
Budgets are available on imdb, at least for some movies
Are budgets in USD?
Other stuff to get from imdb: technical details, etc.
# Log
* Adding mpaa_ratings improved the score to 1.97524
* Fixing a few runtime and year values based on imdb data barely bade a difference: 1.99040
* Adding a log(budget) improved the public score 1.99055
* Doing EDA and feature engineering on homepage improved the public score to 2.02096
* Fixing the century of the release year didn't help (actually made it a little worse)
* A stacked model improved the public score by a lot to 2.02996!!!
* Doing EDA and feature engineering on cast dit not improve the public score (overfitting)
* Doing EDA and feature engineering on crew (barely) improved the public score to 2.38779
* Doing EDA and feature engineering on original_language dit not improve the public score (probably because it's highly correlated with spoken_languages)
* Doing EDA and feature engineering on runtime improved the public score to 2.42784
* Doing EDA and feature engineering on release_date improved the public score to 2.46433
* Doing EDA and feature engineering on spoken_languages improved the public score to 2.46744
* Doing EDA and feature engineering on production_countries improved the public score to 2.47048
* Doing EDA and feature engineering on production_companies improved the public score to 2.49865
* Created a set of utilities for EDA and feature engineering for all dictionary columns, it should make the next columns much faster.
* Doing EDA and feature engineering on genres improved the public score to 2.59730
* Doing EDA and feature engineering on belongs_to_collection improved the public score to 2.67052
* A baseline based only on budget and popularity, with a simple lasso model, resulted in a 2.68084 public score
# References
I've learned a lot from these kernels:
[Stacked Regressions to predict House Prices] by Serigne - how to stack models
[1]: https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

In [1]:
# Import
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import os
import collections
import time
from sklearn.preprocessing import LabelEncoder
print(os.listdir("../input"))

['tmdb-box-office-prediction', 'imdb-web-scraping-4']


In [2]:
# Load files
train = pd.read_csv("../input/tmdb-box-office-prediction/train.csv")
test = pd.read_csv("../input/tmdb-box-office-prediction/test.csv")
imdb = pd.read_csv("../input/imdb-web-scraping-4/imdb.csv")
train.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [3]:
imdb.head()

,imdb_id,imdb_year,imdb_release,imdb_budget,imdb_runtime,imdb_country,mpaa_rating
0,tt2637294,2015,20 February 2015 (USA)\n,"\nBudget:$14,000,000\n (estimated)\n",93.0,USA,Rated R for crude sexual content and language ...
1,tt0368933,2004,11 August 2004 (USA)\n,"\nBudget:$40,000,000\n (estimated)\n",113.0,USA,NaN
2,tt2582802,2014,15 October 2014 (Philippines)\n,"\nBudget:$3,300,000\n (estimated)\n",106.0,USA,Rated R for strong language including some sex...
3,tt1821480,2012,9 March 2012 (India)\n,NaN,122.0,India,NaN
4,tt1380152,2009,5 February 2009 (South Korea)\n,NaN,118.0,South Korea,NaN


In [4]:
# Combine train and test to allow feature engineering on the combined data. Store the ids and the revenue for future use
train_rows = train.shape[0]
y_train = np.log1p(train["revenue"]).values
train_ID = train['id']
test_ID = test['id']

all_data = pd.concat([train, test], sort=False)
all_data.drop(['revenue'], axis=1, inplace=True)
all_data.drop(['id'], axis=1, inplace=True)

# Combine the result with the imdb data
all_data = pd.merge(all_data, imdb, how="left", on="imdb_id")
all_data.head()

,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,imdb_year,imdb_release,imdb_budget,imdb_runtime,imdb_country,mpaa_rating
0,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",2015,20 February 2015 (USA)\n,"\nBudget:$14,000,000\n (estimated)\n",93.0,USA,Rated R for crude sexual content and language ...
1,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",2004,11 August 2004 (USA)\n,"\nBudget:$40,000,000\n (estimated)\n",113.0,USA,NaN
2,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",2014,15 October 2014 (Philippines)\n,"\nBudget:$3,300,000\n (estimated)\n",106.0,USA,Rated R for strong language including some sex...
3,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",2012,9 March 2012 (India)\n,NaN,122.0,India,NaN
4,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",2009,5 February 2009 (South Korea)\n,NaN,118.0,South Korea,NaN


# EDA and Feature Engineering

In [5]:
# Several columns (e.g. genres) are lists of values - split them to dictionaries for easier processing
import ast
for c in ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 
          'Keywords', 'cast', 'crew']:
    all_data[c] = all_data[c].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x))

In [6]:
# The EDA and feature engineering for the dict columns seems similar. Create a few basic common utilities
def print_few_values(col_name):
    print("Sample values for", col_name)
    all_data[col_name].head(5).apply(lambda x: print(x))
    
def dictionary_sizes(col_name):
    return (all_data[col_name].apply(lambda x: len(x)).value_counts())

def print_dictionary_sizes(col_name):
    print("\n===================================================")
    print("Distribution of sizes for", col_name)
    print(dictionary_sizes(col_name))
    
# returns a list of tuples of names for a given row of a column
def dict_name_list(d, name="name"):
    return ([i[name] for i in d] if d != {} else [])

# returns a list of tuples of the (id,name) pairs for a given row of a column
def dict_id_name_list(d, name="name"):
    return ([(i["id"],i[name]) for i in d] if d != {} else [])

# returns a list of names for a given column
def col_name_list(col_name, name="name"):
    # Get a list of lists of names
    name_list_list = list(all_data[col_name].apply(lambda x: dict_name_list(x, name)).values)
    # Merge into 1 list
    return ([i for name_list in name_list_list for i in name_list])

# returns a list of tuples of the (id,name) pairs for a given column
def col_id_name_list(col_name, name="name"):
    # Get a list of lists of (id,name) tuples
    tuple_list_list = list(all_data[col_name].apply(lambda x: dict_id_name_list(x, name)).values)
    # Merge into 1 list
    return ([i for tuple_list in tuple_list_list for i in tuple_list])

def get_names_counter(col_name, name="name"):
    name_list = col_name_list(col_name, name)
    return (collections.Counter(name_list))
    
def print_top_names(col_name, name="name"):
    print("\n===================================================")
    print("Top {0}s for {1}".format(name, col_name))
    c = get_names_counter(col_name, name)
    print(c.most_common(20))
    
def EDA_dict(col_name):
    print_few_values(col_name)
    print_dictionary_sizes(col_name)
    print_top_names(col_name)
    
def add_dict_size_column(col_name):
    all_data[col_name + "_size"] = all_data[col_name].apply(lambda x: len(x) if x != {} else 0)

def add_dict_id_column(col_name):
    c = col_name + "_id"
    all_data[c] = all_data[col_name].apply(lambda x: x[0]["id"] if x != {} else 0)
    all_data[c] = all_data[c].astype("category")

# for each of the top values in the dictionary, add an column indicating if the row belongs to it
def add_dict_indicator_columns(col_name):
    c = get_names_counter(col_name)
    top_names = [x[0] for x in c.most_common(20)]
    for name in top_names:
        all_data[col_name + "_" + name] = all_data[col_name].apply(lambda x: name in dict_name_list(x))
        
def drop_column(col_name):
    all_data.drop([col_name], axis=1, inplace=True)
    
def feature_engineer_dict(col_name):
    add_dict_size_column(col_name)
    max_size = dictionary_sizes(col_name).index.max()
    if max_size == 1:
        add_dict_id_column(col_name)
    else:
        add_dict_indicator_columns(col_name)
    drop_column(col_name)
    
def encode_column(col_name):
    lbl = LabelEncoder()
    lbl.fit(list(all_data[col_name].values)) 
    all_data[col_name] = lbl.transform(list(all_data[col_name].values))

# belongs_to_collection

In [7]:
col_name = "belongs_to_collection"
# EDA
EDA_dict(col_name)

Sample values for belongs_to_collection
[{'id': 313576, 'name': 'Hot Tub Time Machine Collection', 'poster_path': '/iEhb00TGPucF0b4joM1ieyY026U.jpg', 'backdrop_path': '/noeTVcgpBiD48fDjFVic1Vz7ope.jpg'}]
[{'id': 107674, 'name': 'The Princess Diaries Collection', 'poster_path': '/wt5AMbxPTS4Kfjx7Fgm149qPfZl.jpg', 'backdrop_path': '/zSEtYD77pKRJlUPx34BJgUG9v1c.jpg'}]
{}
{}
{}

Distribution of sizes for belongs_to_collection
0    5917
1    1481
Name: belongs_to_collection, dtype: int64

Top names for belongs_to_collection
[('James Bond Collection', 26), ('Friday the 13th Collection', 12), ('Pokémon Collection', 9), ('The Fast and the Furious Collection', 8), ('The Pink Panther (Original) Collection', 8), ('Star Wars Collection', 8), ('Halloween Collection', 8), ('Harry Potter Collection', 8), ('A Nightmare on Elm Street Collection', 8), ('Rocky Collection', 7), ('Police Academy Collection', 7), ('Saw Collection', 7), ('The Muppet Collection', 6), ('X-Men Collection', 6), ('Resident Evil C

In [8]:
# Feature engineering
feature_engineer_dict(col_name)

# genres

In [9]:
col_name="genres"
EDA_dict(col_name)

Sample values for genres
[{'id': 35, 'name': 'Comedy'}]
[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 10749, 'name': 'Romance'}]
[{'id': 18, 'name': 'Drama'}]
[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'name': 'Drama'}]
[{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}]

Distribution of sizes for genres
2    2379
3    2208
1    1488
4     967
5     280
6      48
0      23
7       4
8       1
Name: genres, dtype: int64

Top names for genres
[('Drama', 3676), ('Comedy', 2605), ('Thriller', 1869), ('Action', 1735), ('Romance', 1435), ('Adventure', 1116), ('Crime', 1084), ('Science Fiction', 744), ('Horror', 735), ('Family', 675), ('Fantasy', 628), ('Mystery', 550), ('Animation', 382), ('History', 295), ('Music', 267), ('War', 243), ('Documentary', 221), ('Western', 117), ('Foreign', 84), ('TV Movie', 1)]


In [10]:
# Feature engineering
feature_engineer_dict(col_name)

# production_companies

In [11]:
col_name="production_companies"
EDA_dict(col_name)

Sample values for production_companies
[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'United Artists', 'id': 60}, {'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8411}]
[{'name': 'Walt Disney Pictures', 'id': 2}]
[{'name': 'Bold Films', 'id': 2266}, {'name': 'Blumhouse Productions', 'id': 3172}, {'name': 'Right of Way Films', 'id': 32157}]
{}
{}

Distribution of sizes for production_companies
1     1905
2     1786
3     1373
4      785
5      430
0      414
6      295
7      156
8       91
9       74
11      24
10      22
12       8
16       8
13       6
14       5
19       3
15       3
17       2
18       2
21       2
24       1
20       1
22       1
26       1
Name: production_companies, dtype: int64

Top names for production_companies
[('Warner Bros.', 491), ('Universal Pictures', 463), ('Paramount Pictures', 393), ('Twentieth Century Fox Film Corporation', 341), ('Columbia Pictures', 236), ('Metro-Goldwyn-Mayer (MGM)', 207), ('New Line Cinema', 198), ('Touchstone Pictures', 158), 

In [12]:
# Feature engineering
feature_engineer_dict(col_name)

# production_countries

In [13]:
col_name="production_countries"
EDA_dict(col_name)

Sample values for production_countries
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_3166_1': 'IN', 'name': 'India'}]
[{'iso_3166_1': 'KR', 'name': 'South Korea'}]

Distribution of sizes for production_countries
1     5464
2     1227
3      361
0      157
4      128
5       42
6       11
8        3
9        2
7        1
10       1
12       1
Name: production_countries, dtype: int64

Top names for production_countries
[('United States of America', 5617), ('United Kingdom', 917), ('France', 570), ('Germany', 411), ('Canada', 323), ('India', 220), ('Italy', 160), ('Japan', 157), ('Australia', 148), ('Spain', 139), ('Russia', 132), ('China', 99), ('Hong Kong', 96), ('Belgium', 64), ('Ireland', 62), ('South Korea', 58), ('Sweden', 50), ('Mexico', 44), ('Netherlands', 43), ('Denmark', 40)]


In [14]:
# Feature engineering
feature_engineer_dict(col_name)

# spoken_languages

In [15]:
col_name="spoken_languages"
EDA_dict(col_name)

Sample values for spoken_languages
[{'iso_639_1': 'en', 'name': 'English'}]
[{'iso_639_1': 'en', 'name': 'English'}]
[{'iso_639_1': 'en', 'name': 'English'}]
[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'hi', 'name': 'हिन्दी'}]
[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]

Distribution of sizes for spoken_languages
1    5226
2    1319
3     502
4     178
5      73
0      62
6      25
7       6
8       4
9       3
Name: spoken_languages, dtype: int64

Top names for spoken_languages
[('English', 6449), ('Français', 711), ('Español', 559), ('Deutsch', 419), ('Pусский', 364), ('Italiano', 321), ('日本語', 204), ('普通话', 163), ('हिन्दी', 160), ('', 114), ('العربية', 102), ('Português', 100), ('广州话 / 廣州話', 89), ('한국어/조선말', 78), ('Polski', 69), ('Latin', 66), ('Magyar', 64), ('Český', 57), ('ภาษาไทย', 55), ('தமிழ்', 52)]


In [16]:
# Feature engineering
feature_engineer_dict(col_name)

# release_date

In [17]:
# Check for nulls
all_data.loc[all_data["release_date"].isnull()]
# There is 1 movie w/o a release date. Looking it up in imdb by imdb_id = tt0210130, it was release in march 2000
all_data.loc[all_data["release_date"].isnull(), "release_date"] = "05/01/2000"
# Parse the string to a date
all_data["release_date"] = pd.to_datetime(all_data["release_date"])
# Create columns for each part of the date
all_data["release_date_weekday"] = all_data["release_date"].dt.weekday.astype(int)
all_data["release_date_month"] = all_data["release_date"].dt.month.astype(int)
all_data["release_date_year"] = all_data["release_date"].dt.year.astype(int)
# The year is formatted as yy as opposed to yyyy, and therefore the century is sometimes incorrect.
all_data["release_date_year"] = np.where(all_data["release_date_year"]>2019, all_data["release_date_year"]-100, all_data["release_date_year"])
# Compare with imdb
all_data["imdb_year"]=all_data["imdb_year"].astype(int)
# Compare the values
print(all_data[abs(all_data["release_date_year"]-all_data["imdb_year"])>2][["imdb_id","release_date_year","imdb_year"]])
# There are 4 movies from the 20th century that appear as 21st century due to the issue above. Fix them.
all_data.loc[abs(all_data["release_date_year"]-all_data["imdb_year"])>2, "release_date_year"] = all_data["imdb_year"]
drop_column("release_date")
drop_column("imdb_year")

        imdb_id  release_date_year  imdb_year
605   tt0004972               2015       1915
2226  tt0006333               2016       1916
3812  tt0005078               2015       1915
5307  tt0009369               2018       1918


# runtime

In [18]:
all_data.loc[all_data["runtime"].isnull(), "imdb_id"]
all_data.loc[all_data["runtime"]==0, "imdb_id"]
# A few movies don't have runtime. Get their imdb_id to look them up on imdb.
all_data.loc[all_data["runtime"]==0, "runtime"] = all_data["imdb_runtime"]
all_data.loc[all_data["runtime"].isnull(), "runtime"] = all_data["imdb_runtime"]
# Most of them have values in imdb. Found one more online.Filling the remaining ones with the column average 
all_data["runtime"].fillna(all_data["runtime"].mean(), inplace=True)
drop_column("imdb_runtime")

# original_language

In [19]:
encode_column("original_language")

In [20]:
# Fix Gender
# Both cast and crew have lots of entries with gender == 0 (i.e. unknown). 
# Use the rest of the entries to decide if a first name is male or female, and categorize the unknown ones by that

# Create a DataFrame of all the names in the cast and crew columns and how many times they appear as each gender
def get_names():
    tuple_list_list = list(all_data["cast"].apply(lambda x: [(cm["name"].split(" ")[0],cm["gender"]) for cm in x]).values)
    tuple_list_list.extend(list(all_data["crew"].apply(lambda x: [(cm["name"].split(" ")[0],cm["gender"]) for cm in x]).values))
    tuple_list = [i for tuple_list in tuple_list_list for i in tuple_list]
    names = set()
    c = [None] * 3
    for gender in range (0, 3):
        t = [i[0] for i in tuple_list if i[1] == gender]
        names.update(set(t))
        c[gender] = collections.Counter(t)
    names_df = pd.DataFrame(data = list(names), columns=["name"])
    names_df["female_count"] = names_df["name"].apply(lambda x: c[1][x])
    names_df["male_count"] = names_df["name"].apply(lambda x: c[2][x])
    names_df["unknown_count"] = names_df["name"].apply(lambda x: c[0][x])
    names_df["total_count"] = names_df.apply(lambda x: x["female_count"]+x["male_count"]+x["unknown_count"], axis=1)
    return (names_df)

names_df = get_names()

# Classify each name with a gender (or leave as 0 if there is not enough data to decide or the name is unisex)
def ClassifyName(row):
    fcount = row["female_count"]
    mcount = row["male_count"]
    cl = "TBD"
    gender = 0
    if (fcount + mcount < 5):
        cl = "too few names"
    elif (fcount == 0):
        gender = 2
        cl = "high confidence"
    elif (mcount == 0):
        gender = 1
        cl = "high confidence"
    else:  # both are > 0
        # If a name is 90+% male or female, even if unisex, we'll bet on the majority
        if (mcount / float(fcount) < 0.1):
            gender = 1
            cl = "unisex - pick one"
        elif (fcount / float(mcount) < 0.1):
            gender = 2
            cl = "unisex - pick one"
        else: #unisex, no sex more than 90% - leave as undefined
            cl = "unisex - TBD"
    return [gender, cl]

names_df = names_df.merge(names_df.apply(lambda x: ClassifyName(x), axis=1, result_type="expand"), left_index=True, right_index=True)
names_df.rename(columns={0:"gender", 1:"classification"}, inplace=True)
# pd.pivot_table(names_df, index=["classification"], values="total_count", aggfunc=[np.sum, "count"])

# Create a dictionary that maps each name that as male or female (to not add names with undefined gender)
names_to_gender = dict()
def update_names_to_gender(row):
    if (row["gender"] > 0):
        names_to_gender[row["name"]] = row["gender"]
j = names_df.apply(lambda x: update_names_to_gender(x), axis=1)
# names_to_gender

# Method to fix the entries with gender == 0 (in cast or crew)
def fix_unknown_gender_row(row):
    for cm in row:
        if cm["gender"] == 0:
            name = cm["name"]
            first_name = name.split(" ")[0]
            if (first_name in names_to_gender):
                cm["gender"] = names_to_gender[first_name]
    
def fix_unknown_gender_col(col_name):
    all_data[col_name].apply(lambda x: fix_unknown_gender_row(x))

# crew

In [21]:
col_name = "crew"
EDA_dict(col_name)
# The crew column is different than others as it has much more properties. Let's look at jobs
print_top_names(col_name, name="job")

Sample values for crew
[{'credit_id': '59ac067c92514107af02c8c8', 'department': 'Directing', 'gender': 0, 'id': 1449071, 'job': 'First Assistant Director', 'name': 'Kelly Cantley', 'profile_path': None}, {'credit_id': '52fe4ee7c3a36847f82afad7', 'department': 'Directing', 'gender': 2, 'id': 3227, 'job': 'Director', 'name': 'Steve Pink', 'profile_path': '/myHOgo8mQSCiCAZNGMRdHVr03jr.jpg'}, {'credit_id': '5524ed25c3a3687ded000d88', 'department': 'Writing', 'gender': 2, 'id': 347335, 'job': 'Writer', 'name': 'Josh Heald', 'profile_path': '/pwXJIenrDMrG7t3zNfLvr8w1RGU.jpg'}, {'credit_id': '5524ed2d925141720c001128', 'department': 'Writing', 'gender': 2, 'id': 347335, 'job': 'Characters', 'name': 'Josh Heald', 'profile_path': '/pwXJIenrDMrG7t3zNfLvr8w1RGU.jpg'}, {'credit_id': '5524ed3d92514166c1004a5d', 'department': 'Production', 'gender': 2, 'id': 57822, 'job': 'Producer', 'name': 'Andrew Panay', 'profile_path': None}, {'credit_id': '5524ed4bc3a3687df3000dd2', 'department': 'Production', 

In [22]:
fix_unknown_gender_col(col_name)

def get_most_common_gender(row):
    genders = [mc["gender"] for mc in row]
    if (genders == []):
        return -1
    return (collections.Counter(genders).most_common(1)[0][0])
        
# Split by job
top_jobs = [x[0] for x in get_names_counter(col_name, "job").most_common(10)]
for job in top_jobs:
    c = "crew_" + job
    print("-------------\n",job,"\n-------------")
    all_data[c] = all_data[col_name].apply(lambda x: [crew_member for crew_member in x if crew_member["job"]==job])
    all_data[c + "_common_gender"] = all_data[c].apply(lambda x: get_most_common_gender(x))
    EDA_dict(c)
    feature_engineer_dict(c)
drop_column(col_name)

-------------
 Producer 
-------------
Sample values for crew_Producer
[{'credit_id': '5524ed3d92514166c1004a5d', 'department': 'Production', 'gender': 2, 'id': 57822, 'job': 'Producer', 'name': 'Andrew Panay', 'profile_path': None}]
[{'credit_id': '52fe43fe9251416c7502564f', 'department': 'Production', 'gender': 1, 'id': 8851, 'job': 'Producer', 'name': 'Whitney Houston', 'profile_path': '/69ouDnXnmklYPr4sMJXWKYz81AL.jpg'}, {'credit_id': '52fe43fe9251416c75025655', 'department': 'Production', 'gender': 2, 'id': 38415, 'job': 'Producer', 'name': 'Mario Iscovich', 'profile_path': None}, {'credit_id': '52fe43fe9251416c75025649', 'department': 'Production', 'gender': 1, 'id': 59973, 'job': 'Producer', 'name': 'Debra Martin Chase', 'profile_path': None}]
[{'credit_id': '52fe4ef7c3a36847f82b3fd9', 'department': 'Production', 'gender': 2, 'id': 60864, 'job': 'Producer', 'name': 'David Lancaster', 'profile_path': None}, {'credit_id': '52fe4ef7c3a36847f82b3fdf', 'department': 'Production', 'ge

# cast

In [23]:
col_name = "cast"
# EDA_dict(col_name)

In [24]:
fix_unknown_gender_col(col_name)

# Split by gender
genders = [x[0] for x in get_names_counter(col_name, "gender").most_common(2)]
for gender in genders:
    c = "cast_gender_" + str(gender)
    all_data[c] = all_data[col_name].apply(lambda x: [cast_member for cast_member in x if cast_member["gender"]==gender])
    #print("-------------\n",gender,"\n-------------")
    EDA_dict(c)
    feature_engineer_dict(c)
drop_column(col_name)

Sample values for cast_gender_2
[{'cast_id': 4, 'character': 'Lou', 'credit_id': '52fe4ee7c3a36847f82afae7', 'gender': 2, 'id': 52997, 'name': 'Rob Corddry', 'order': 0, 'profile_path': '/k2zJL0V1nEZuFT08xUdOd3ucfXz.jpg'}, {'cast_id': 5, 'character': 'Nick', 'credit_id': '52fe4ee7c3a36847f82afaeb', 'gender': 2, 'id': 64342, 'name': 'Craig Robinson', 'order': 1, 'profile_path': '/tVaRMkJXOEVhYxtnnFuhqW0Rjzz.jpg'}, {'cast_id': 6, 'character': 'Jacob', 'credit_id': '52fe4ee7c3a36847f82afaef', 'gender': 2, 'id': 54729, 'name': 'Clark Duke', 'order': 2, 'profile_path': '/oNzK0umwm5Wn0wyEbOy6TVJCSBn.jpg'}, {'cast_id': 7, 'character': 'Adam Jr.', 'credit_id': '52fe4ee7c3a36847f82afaf3', 'gender': 2, 'id': 36801, 'name': 'Adam Scott', 'order': 3, 'profile_path': '/5gb65xz8bzd42yjMAl4zwo4cvKw.jpg'}, {'cast_id': 8, 'character': 'Hot Tub Repairman', 'credit_id': '52fe4ee7c3a36847f82afaf7', 'gender': 2, 'id': 54812, 'name': 'Chevy Chase', 'order': 4, 'profile_path': '/svjpyYtPwtjvRxX9IZnOmOkhDOt.j

# homepage

In [25]:
# Add an indicator for lack of homepage
all_data["no_homepage"] = all_data["homepage"].isnull()
print(all_data["no_homepage"].value_counts())
# Create a list of all homepages that appear more than once
multiple_homepages = all_data["homepage"].value_counts()
multiple_homepages = multiple_homepages[multiple_homepages>1].index.tolist()
print("There are ", len(multiple_homepages), "homepages which appear more than once")
# All the unique homepages are not interesting, replace them with null
all_data.loc[all_data["homepage"].isin(multiple_homepages) == False, "homepage"] = "" 
# Encode the strings
encode_column("homepage")

True     5032
False    2366
Name: no_homepage, dtype: int64
There are  26 homepages which appear more than once


# budget

In [26]:
# all_data["budget"].value_counts().sort_index()
# all_data[(all_data.budget>0) & (all_data.budget<1000)][["budget","release_date_year","title"]]
all_data["no_budget"] =(all_data["budget"] == 0)
# all_data[(all_data.budget>0) & (all_data.budget<1000), "budget"] = all_data["budget"] * 1000000
# all_data.head()
all_data["log_budget"]=np.log1p(all_data["budget"])

# mpaa_rating

In [27]:
# Ratings have the following format: Rated <rating> for <reason>
# Remove the word "Rating at the beginning"
all_data["mpaa_rating"] = all_data["mpaa_rating"].apply(lambda s: s.split(" ", 1)[1] if isinstance(s, str) else s)
# Set the reason to be the sentence after ""<rating> for"
all_data["mpaa_rating_reason"] = all_data["mpaa_rating"].apply(lambda s: s.split(" ", 2)[2] if isinstance(s, str) else s)
# Set the rating to be the first word
all_data["mpaa_rating"] = all_data["mpaa_rating"].apply(lambda s: s.split(" ", 1)[0] if isinstance(s, str) else s)
all_data["mpaa_rating"].value_counts()

R        2473
PG-13    1578
PG        654
NC-17      12
Name: mpaa_rating, dtype: int64

In [28]:
encode_column("mpaa_rating")

In [29]:
def SplitReason(r):
    s = r.strip().replace("  ", " ").replace(", and for ", ", ").replace(", and ", ", ")
    rl = s.rsplit(" and ", 1)
    rl2 = rl[0].split(", ")
    if len(rl)>1:
        rl2.append(rl[1])
    return (rl2)

# Split the reasons, create an indicator column for the top reasons
col_name = "mpaa_rating_reason"
all_data[col_name] = all_data[col_name].apply(lambda s: SplitReason(s) if isinstance(s, str) else [])
reason_list_list = list(all_data[col_name].values)
top_reasons = [reason[0] for reason in collections.Counter([i for reason_list in reason_list_list for i in reason_list]).most_common(100)]
for reason in top_reasons:
    all_data[col_name + "_" + reason] = all_data[col_name].apply(lambda x: reason in x)
all_data.head()
drop_column(col_name)

In [30]:
# Drop unneeded columns

# Drop all columns which haven't been handled yet. We'll gradually add them back later on
all_data.drop(['imdb_id'], axis=1, inplace=True)
all_data.drop(['original_title'], axis=1, inplace=True)
all_data.drop(['overview'], axis=1, inplace=True)
all_data.drop(['poster_path'], axis=1, inplace=True)
all_data.drop(['status'], axis=1, inplace=True)
all_data.drop(['tagline'], axis=1, inplace=True)
all_data.drop(['title'], axis=1, inplace=True)
all_data.drop(['Keywords'], axis=1, inplace=True)

drop_column("imdb_release")
drop_column("imdb_budget")
drop_column("imdb_country")

In [31]:
# Split back to train and test
train = all_data[:train_rows]
test = all_data[train_rows:]
train.describe()

,budget,homepage,original_language,popularity,runtime,mpaa_rating,belongs_to_collection_size,genres_size,production_companies_size,production_countries_size,spoken_languages_size,release_date_weekday,release_date_month,release_date_year,crew_Producer_common_gender,crew_Producer_size,crew_Executive Producer_common_gender,crew_Executive Producer_size,crew_Director_common_gender,crew_Director_size,crew_Screenplay_common_gender,crew_Screenplay_size,crew_Editor_common_gender,crew_Editor_size,crew_Casting_common_gender,crew_Casting_size,crew_Director of Photography_common_gender,crew_Director of Photography_size,crew_Original Music Composer_common_gender,crew_Original Music Composer_size,crew_Art Direction_common_gender,crew_Art Direction_size,crew_Production Design_common_gender,crew_Production Design_size,cast_gender_2_size,cast_gender_1_size,log_budget
count,3.000000e+03,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.00000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.00000,3000.000000
mean,2.253133e+07,0.110000,11.475000,8.463274,108.251332,2.995667,0.201333,2.503667,2.698333,1.326333,1.452333,3.255667,6.775333,1999.646333,1.193333,2.003667,0.232333,1.15300,1.832333,1.075000,0.673000,0.998667,0.975333,0.941333,0.072667,0.827667,1.060667,0.762667,0.645333,0.649000,0.093333,0.607000,0.405667,0.550000,12.774000,5.76900,11.878667
std,3.702609e+07,1.488614,5.321981,12.104000,21.018312,0.953914,0.401063,1.117021,2.014121,0.752349,0.887688,1.325979,3.409115,15.570401,1.229279,1.727999,1.431213,1.86532,0.522474,0.646669,1.431347,1.075043,1.299470,0.775516,1.125674,1.045147,1.344714,0.528298,1.459082,0.642342,1.370370,0.995765,1.406803,0.544303,10.946729,5.87337,7.441843
min,0.000000e+00,0.000000,1.000000,0.000001,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1915.000000,-1.000000,0.000000,-1.000000,0.00000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000e+00,0.000000,10.000000,4.018053,94.000000,2.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,4.000000,1993.000000,0.000000,1.000000,-1.000000,0.00000,2.000000,1.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,6.000000,2.00000,0.000000
50%,8.000000e+06,0.000000,10.000000,7.374861,104.000000,3.000000,0.000000,2.000000,2.000000,1.000000,1.000000,4.000000,7.000000,2004.000000,2.000000,2.000000,-1.000000,0.00000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,0.000000,1.000000,2.000000,1.000000,2.000000,1.000000,-1.000000,0.000000,0.000000,1.000000,10.000000,4.00000,15.894952
75%,2.900000e+07,0.000000,10.000000,10.890983,118.000000,4.000000,0.000000,3.000000,4.000000,1.000000,2.000000,4.000000,10.000000,2011.000000,2.000000,3.000000,2.000000,2.00000,2.000000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,15.000000,7.00000,17.182806
max,3.800000e+08,25.000000,43.000000,294.337037,338.000000,4.000000,1.000000,7.000000,17.000000,8.000000,9.000000,6.000000,12.000000,2017.000000,2.000000,12.000000,2.000000,18.00000,2.000000,30.000000,2.000000,12.000000,2.000000,8.000000,2.000000,6.000000,2.000000,6.000000,2.000000,7.000000,2.000000,11.000000,2.000000,4.000000,102.000000,97.00000,19.755682


In [32]:
# Create a model
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv=kf))
    return(rmse)

def eval_model(model, name):
    start_time = time.time()
    score = rmsle_cv(model)
    print("{} score: {:.4f} ({:.4f}),     execution time: {:.1f}".format(name, score.mean(), score.std(), time.time()-start_time))

In [33]:
mod_lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.005, random_state=1))
eval_model(mod_lasso, "lasso")
mod_enet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
eval_model(mod_enet, "enet")
mod_cat = CatBoostRegressor(iterations=10000, learning_rate=0.01,
                            depth=5, eval_metric='RMSE',
                            colsample_bylevel=0.7, random_seed = 17, silent=True,
                            bagging_temperature = 0.2, early_stopping_rounds=200)
eval_model(mod_cat, "cat")
mod_gboost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=5, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state=5)
eval_model(mod_gboost, "gboost")
mod_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state=7, nthread=-1)
eval_model(mod_xgb, "xgb")
mod_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=8,
                              learning_rate=0.05, n_estimators=650,
                              max_bin=58, bagging_fraction=0.80,
                              bagging_freq=5, feature_fraction=0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf=7, min_sum_hessian_in_leaf=11)
eval_model(mod_lgb, "lgb")

lasso score: 2.1649 (0.0994),     execution time: 3.5
enet score: 2.2489 (0.0796),     execution time: 5.6
cat score: 2.1007 (0.0825),     execution time: 683.6
gboost score: 2.0868 (0.0816),     execution time: 80.0
xgb score: 2.1116 (0.0889),     execution time: 72.0
lgb score: 2.0561 (0.0972),     execution time: 2.3


In [34]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)
    
mod_stacked = StackingAveragedModels(base_models = (mod_cat, mod_xgb, mod_gboost, mod_lgb), meta_model = mod_lasso)
eval_model(mod_stacked, "stacked")

stacked score: 2.0375 (0.0962),     execution time: 3922.0


In [35]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def predict(model):
    model.fit(train.values, y_train)
    train_pred = model.predict(train.values)
    pred = np.expm1(model.predict(test.values))
    print(rmsle(y_train, train_pred))
    return (pred)

In [36]:
# Predict
# Start with a simple model
prediction = predict(mod_lasso)
prediction = predict(mod_enet)
prediction = predict(mod_xgb)
prediction = predict(mod_gboost)
prediction = predict(mod_lgb)
prediction = predict(mod_stacked)

2.087295127866839
1.9953493305153052
0.9266918192823541
1.3087767809939983
1.465247782384994
1.4063740463752452


In [37]:
# Submit
submission = pd.DataFrame()
submission['id'] = test_ID
submission['revenue'] = prediction
submission.to_csv('submission.csv', index=False)

In [38]:
# Log                     Estimate Public score
# baseline:               2.6454   2.68084    v2
# belong_to_collection:   2.6437   2.67052    v5
# genres:                 2.5476   2.59730    v6
# production_companies:   2.4396   2.49865    v7
# production_countries:   2.4340   2.47048    v8
# spoken_languages:       2.4316   2.46744    v9
# release_date:           2.4274   2.46433    v10
# runtime:                2.4034   2.42784    v11
# original_language       2.4050   2.42811    v12 - no improvement
# crew                    2.3649   2.38779    v13
# cast                    2.3714   2.40004    v14 - no improvement. Maybe it means that we are overfitting with so many columns.
# stacked model           2.0624   2.02996    v15
# Fix years (century)     2.0632   2.03133    v16 - didn't help
# homepage                2.0623   2.02096    v17
# added no_budget         2.0551   2.02131    v19 - didn't help
# added log(budget)       2.0417   1.99055    v20
# filled runtime from imdb2.0424   1.99148    v21 - didn't help
# fixed century of 4 items2.0448   1.99040    v22
# add mpaa_rating         2.0333   1.97524    v23
# add mpaa_rating_reason  2.0473   1.97890    v24
# add cat model, rem lasso2.1055   2.01712    v25
# bring back lasso        2.0402   1.97147    v26
# manual model param tune 2.0367   1.96204    v27